<a href="https://colab.research.google.com/github/ahairshi/dsf-redis/blob/main/Write_a_python_script_to_convert_any_curl_into_lo_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

It's common practice in performance testing to want to convert existing HTTP requests, like those captured as **cURL** commands, into a **Locust** load test script (a `locustfile.py`). This allows you to easily reproduce and scale up those requests for stress testing.

While there isn't a single universal tool that perfectly converts *any* cURL command into a full, production-ready Locust script, the core idea is to parse the cURL command and map its components (URL, method, headers, data) to the equivalent **`HttpUser`** methods in Locust.

Here is a conceptual Python script that demonstrates how you might manually parse a simple cURL command and create a basic Locust script template. For complex cURL commands, you might consider using a dedicated library like `curlify` or an online converter to first get the Python `requests` code, which is then easier to adapt for Locust.

### Python Script Template (`curl_to_locust.py`)

This script takes a *simplified* cURL command string and generates a basic `locustfile.py`.

In [ ]:
import re

# --- The function to parse cURL and generate Locust code ---

def generate_locust_script(curl_command):
    """
    Parses a simplified cURL command and generates a basic Locust script string.

    This function handles:
    - URL
    - Request Method (-X)
    - Headers (-H)
    - Data/Payload (-d)

    NOTE: This is a basic parser and may not handle all cURL complexities
    (e.g., escaping, cookies, form data, different quote types).
    For production, consider using a robust library or a manual conversion
    from a tool like curlconverter.
    """

    # 1. Extract the URL (assuming it's the first non-flag argument)
    url_match = re.search(r'curl\s+[\'"]?([^\s"\']+)[\'"]?', curl_command, re.IGNORECASE)
    url = url_match.group(1) if url_match else "http://your-target-host/"

    # Determine the base host for Locust (needs to be configured when running Locust)
    # This part is just for clarity; Locust's HttpUser base URL is set when starting the test.
    # We'll just extract the path for the task.
    path = "/"
    if url:
        try:
            from urllib.parse import urlparse
            parsed_url = urlparse(url)
            path = parsed_url.path if parsed_url.path else "/"
            # The netloc (host:port) should be passed to the Locust 'host' argument
            # when running the test. We'll set a placeholder in the script.
            host_placeholder = f"http://{parsed_url.netloc}"
        except ImportError:
            # Fallback for systems without urllib.parse
            host_placeholder = "http://your-target-host:port"

    # 2. Extract Request Method
    method_match = re.search(r'-X\s+(\w+)', curl_command, re.IGNORECASE)
    method = method_match.group(1).lower() if method_match else 'get'

    # 3. Extract Headers
    headers = {}
    # Regex to find all -H "Header: Value" or -H 'Header: Value'
    header_matches = re.findall(r"-H\s+['\"]([^'\"]+:\s*[^'\"]+)['\"]", curl_command)
    for h in header_matches:
        key, value = h.split(':', 1)
        headers[key.strip()] = value.strip()

    headers_str = f"{{ {', '.join([f'\"{k}\": \"{v}\"' for k, v in headers.items()])} }}" if headers else "{}"

    # 4. Extract Data/Payload (-d)
    data_match = re.search(r"-d\s+['\"]?(.+?)['\"]?(?=\s|$)", curl_command)
    data = data_match.group(1) if data_match else None

    # Attempt to clean up escaped quotes in JSON data
    if data and ('content-type' in [k.lower() for k in headers.keys()] and 'json' in headers.get('Content-Type', '').lower()):
        # For JSON data, we should pass it as the 'json' parameter in Locust/requests
        # after parsing the string, but here we'll keep it as a string for simplicity.
        # A proper implementation would use json.loads() and pass the dict to 'json='
        data = data.replace('\\"', '"')
        data_param = f'json=json.loads(\'{data}\')'
        import_json = 'import json\n'
    elif data:
        # For form data, pass as 'data='
        data_param = f"data='{data}'"
        import_json = ''
    else:
        data_param = ""
        import_json = ''

    # --- Construct the Locust Code ---

    locust_script = f"""
from locust import HttpUser, task, between
{import_json}

class QuickstartUser(HttpUser):
    # Set a placeholder for the host. You should specify this when running Locust:
    # locust -f locustfile.py --host {host_placeholder}
    host = "{host_placeholder}"

    # Wait between 1 and 2.5 seconds after each task
    wait_time = between(1, 2.5)

    @task
    def my_curl_request(self):
        # Original Method: {method.upper()}
        # Original Path: {path}

        # NOTE: For proper load testing, you should define an appropriate
        # 'name' attribute on the request if the URL contains dynamic data.

        self.client.{method}("{path}",
                                 headers={headers_str},
                                 {data_param})

# Example Call:
# self.client.post("/api/users", headers={{"Content-Type": "application/json"}}, json={{"name": "John Doe"}})
"""
    return locust_script.strip()

# --- Example Usage ---

# 1. Example GET command
curl_get = "curl 'https://jsonplaceholder.typicode.com/todos/1' -H 'User-Agent: MyLoadTestClient'"
print("--- GENERATING LOCUST SCRIPT FOR SIMPLE GET ---")
locust_code_get = generate_locust_script(curl_get)
print(locust_code_get)

# 2. Example POST command with JSON data
curl_post = """curl -X POST 'https://jsonplaceholder.typicode.com/posts' \
  -H 'Content-Type: application/json' \
  -H 'Authorization: Bearer mytoken123' \
  -d '{"title": "foo", "body": "bar", "userId": 1}'"""
print("\n" + "="*50 + "\n")
print("--- GENERATING LOCUST SCRIPT FOR POST WITH JSON ---")
locust_code_post = generate_locust_script(curl_post)
print(locust_code_post)

# Optional: Save to file (uncomment to create the file)
# with open("locustfile.py", "w") as f:
#     f.write(locust_code_post)
# print("\nLocust script saved to locustfile.py")

### Explanation of the Locust Code

1.  **`from locust import HttpUser, task, between`**: Imports necessary components from the Locust library.
2.  **`import json`**: Imported conditionally if the request has a JSON payload, as the best practice in Locust/requests is to pass a Python dictionary to the `json=` parameter.
3.  **`class QuickstartUser(HttpUser):`**: Defines the user behavior. All simulated users will run an instance of this class.
      * **`host = "http://your-target-host:port"`**: This is where the base URL of the service you are testing is defined. The path in the task will be appended to this. You can override this when running Locust using the `--host` argument.
      * **`wait_time = between(1, 2.5)`**: This makes the simulated user wait between 1 and 2.5 seconds after executing a task before starting a new one. This is crucial for realistic load testing.
      * **`@task`**: Decorator that marks a method as an action (or "task") that a user will execute.
      * **`self.client.{method}("{path}", ...)`**: This uses the Locust client (which is essentially a wrapper around the Python `requests` library) to make the HTTP call.
          * `{method}` will be `get`, `post`, etc.
          * `"{path}"` is the relative path from the `host`.
          * `headers={headers_str}` passes the HTTP headers.
          * `data='...'` or `json=json.loads('...')` passes the request body/payload.